In [200]:
import numpy as np

In [201]:
def he_initialization(shape):
    n_in = shape[0]
    stddev = np.sqrt(2.0 / n_in)
    return np.random.normal(0, stddev, shape)

#### 1. Build sum layer ($y = \Sigma wx + b$)

In [202]:
class Sum():
    def __init__(self,neuron_no,back_neuron_no): 
        #imagine a 4 -> 3 network , back is 4 , mine is 3 
        # W: [4,3]
        self.W = he_initialization((back_neuron_no,neuron_no))
        # B: [1,3]
        self.B = he_initialization((neuron_no,))
        # GW: [4,3]
        # GB: [1,3]
        # GD: [1,4]
        self.lr 
    
    
    def forward(self,X): 
        # X : [1,4] , W : [4,3] -> [1,3] + [1,3] => [1,3]
        self.out = np.dot(X,self.W) + self.B
        self.X = X
        return self.out
    # frontGD = [1,3]
    def back(self,frontGD):
        # X: [1,4], X.T : [4,1]  dot -> [4,3]
        self.GW =  np.dot(self.X.T,frontGD)/frontGD.shape[0]
        self.GB = np.sum(frontGD,axis=0)/frontGD.shape[0]
        self.GD = np.dot(frontGD,self.W.T)
        self.W = self.W - self.lr * self.GW
        self.B = self.B - self.lr * self.GB
        return self.GD


#### 2. Build ReLU Activation Layer ($y=max(x,0)$)

In [203]:
class ReLU():
    def __init__(self): 
        pass
    
    def forward(self,X): 
        self.out = np.maximum(X,0)
        return self.out
    # frontGD = [1,3]
    def back(self,frontGD):
        self.GD = frontGD.copy()
        self.GD[self.out<=0]=0
        return self.GD

#### 3. Build SoftMax Activation Layer ($y=\frac {e^{x_i}} {\Sigma e^x}$)

In [204]:
class SoftMax():
    def __init__(self): 
        pass
    
    def forward(self,X): 

        e_x = np.exp(X - np.max(X,axis=1,keepdims=True))
        return e_x / e_x.sum(axis=1,keepdims=True)

    def back(self,frontGD):
        self.GD = frontGD.copy()
        return self.GD


#### 4. Build categorical_crossentropy loss function ($y=-\Sigma ylog\hat y$)

In [205]:
class Loss():
    def __init__(self): 
        self.loss= 0
        self.predict=0.0
        
    def calcLoss(self,y,y_hat): 
        self.y=y
        self.y_hat=y_hat
        loss = -np.log(y_hat+1e-10)*y
        self.loss =self.loss+ np.sum(loss)
        self.predict += np.sum(np.argmax(y_hat,axis=1)== np.argmax(y,axis=1))
    def back(self):
        return self.y_hat-self.y

#### 5. Define Dense layer, combination of sum + ReLU activation 

In [206]:
class Dense():
    def __init__(self,neuron_no,back_neuron_no): 
        self.sum = Sum(neuron_no,back_neuron_no)
        self.activate = ReLU()
        
    def forward(self,X): 
        
        return self.activate.forward(self.sum.forward(X))

    # frontGD = [1,3]
    def back(self,frontGD):
        return self.sum.back(self.activate.back(frontGD))

#### 6. Define Output layer, combination of sum + SoftMax activation 

In [207]:
class Output():
    def __init__(self,neuron_no,back_neuron_no): 
        self.sum = Sum(neuron_no,back_neuron_no)
        self.activate = SoftMax()
    def forward(self,X): 
        return self.activate.forward(self.sum.forward(X))

    def back(self,frontGD):
        return self.sum.back(self.activate.back(frontGD))

#### 7. Load the fashion MNIST dataset

In [208]:
import keras

In [209]:
(trainx, trainy), (testx,testy)=keras.datasets.fashion_mnist.load_data()

In [210]:
train_x = np.reshape(trainx,(trainx.shape[0],-1))
test_x = np.reshape(testx,(testx.shape[0],-1))

In [211]:
train_x=train_x/255
test_x=test_x/255

In [212]:
train_x.shape

(60000, 784)

In [213]:
trainy.shape

(60000,)

In [214]:
train_y = keras.utils.to_categorical(trainy)
test_y = keras.utils.to_categorical(testy)

In [215]:
train_y[0:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [216]:
train_x.shape[0]

60000

#### 8. Sequential Neuron Network

In [217]:
def functionChain(funcs,value):
    if not funcs:
        return value
    first_func = funcs[0]
    rest_funcs = funcs[1:]

    return functionChain(rest_funcs, first_func(value))

In [218]:
class SNN():
    def __init__(self): 
        self.layers = []
        self.forward = []
        self.back = []

    def addLayer(self,layer,neuron_no,input_shape=None):
        if len(self.layers) == 0 :
            self.layers.append(layer(neuron_no,input_shape))
        else :
            self.layers.append(layer(neuron_no,self.backnn_no))
        self.backnn_no = neuron_no
    def compile(self,loss=Loss,lr=0.03):
        self.loss=loss
        for i in self.layers:
            i.sum.lr = lr
            self.forward.append(i.forward)
        for i in self.layers[::-1]:
            self.back.append(i.back)
    def summary(self):
        print("Model summary:")
        print("-"*30)
        print("Model Name\tOutput\tParameters")
        for i,layer in enumerate(self.layers):
            print(f"{layer.__class__.__name__}{i+1}\t[batch_size,{layer.sum.B.size }]\t{layer.sum.B.size+layer.sum.W.size}")

    def fit(self,trainx,trainy,validx,validy,epochs=10,batchsize=1,):

        for epoch in range (0,epochs):
            #Train Part
            loss=self.loss()
            i=0
            while i<trainx.shape[0]:
                batchindex = min(i+batchsize,trainx.shape[0])

                loss.calcLoss(trainy[i:batchindex],functionChain(self.forward,trainx[i:batchindex]))
                functionChain(self.back,loss.back())               
                i=batchindex
                
            print("epoch",epoch+1," accuracy score:", loss.predict/trainx.shape[0],"avg loss:",loss.loss/trainx.shape[0])
            #Validation Part
            loss=self.loss()
            j=0
            while j<validx.shape[0]:
                batchindex = min(j+batchsize,validx.shape[0])
                loss.calcLoss(validy[j:batchindex],functionChain(self.forward,validx[j:batchindex]))       
                j=batchindex
            print("epoch",epoch+1," validation accuracy score:", loss.predict/validx.shape[0],"avg validation loss:",loss.loss/validx.shape[0])

In [219]:
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

(60000, 784) (60000, 10) (10000, 784) (10000, 10)


In [220]:
model=SNN()
model.addLayer(Dense,64,input_shape=train_x.shape[1])
model.addLayer(Dense,32)
model.addLayer(Output,10)
model.compile(lr=0.01)
model.summary()


Model summary:
------------------------------
Model Name	Output	Parameters
Dense1	[batch_size,64]	50240
Dense2	[batch_size,32]	2080
Output3	[batch_size,10]	330


#### 9. Train

In [221]:
model.fit(trainx=train_x,trainy=train_y,validx=test_x,validy=test_y,epochs=20,batchsize=100)

epoch 1  accuracy score: 0.6521833333333333 avg loss: 1.0917654923484579
epoch 1  validation accuracy score: 0.7468 avg validation loss: 0.7382853879390856
epoch 2  accuracy score: 0.77955 avg loss: 0.6428361518721978
epoch 2  validation accuracy score: 0.7886 avg validation loss: 0.6066771250651872
epoch 3  accuracy score: 0.8058333333333333 avg loss: 0.556529021019899
epoch 3  validation accuracy score: 0.8089 avg validation loss: 0.5511329112197152


KeyboardInterrupt: 